In [14]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from src.activations import ActivationType
from src.initialiaztion import get_initialization
from src.losses import LossType
import numpy as np

In [15]:
xavier_initialization = get_initialization("xavier")
he_initialization = get_initialization("he")

In [ ]:
class NeuralNetwork:
  def __init__(self, 
               layer_dims: list[int], 
               activations: list[ActivationType], 
               loss_type=LossType, 
               optimizer_type = list[str],
               seed : int = 42):
    self._validate_inputs(layer_dims, activations, loss_type, optimizer_type)
    
    self.layer_dims = layer_dims
    self.activations = activations
    self.loss_type = loss_type
    self.optimizer_type = optimizer_type
    self.seed = seed
    self.params = {}
    
    self._initialize_parameters()
    
  def _initialize_parameters(self):
      L = len(self.layer_dims)
      
      for i in range(1, L):
          D_o =  self.layer_dims[i]
          D_i = self.layer_dims[i - 1]
          act_fnc = self.activations[i - 1]
          
          if act_fnc == "relu":
              self.params[f"W{i}"] = he_initialization((D_o, D_i))
          if act_fnc == "sigmoid" or act_fnc == "softmax":
              self.params[f"W{i}"] = xavier_initialization((D_o, D_i))
          
          self.params[f"b{i}"] = np.zeros((D_o, 1))
          
  def _validate_inputs(self, layer_dims, activations, loss_type, optimizer_type):
    """
    Private helper to validate all inputs before initialization.
    """
    if not isinstance(layer_dims, list):
        raise TypeError(f"layer_dims must be a list, got {type(layer_dims)}")
    
    if not all(isinstance(x, int) for x in layer_dims):
        raise TypeError("All elements in layer_dims must be integers!")

    if not isinstance(activations, list):
          raise TypeError(f"activations must be a list, got {type(activations)}")

    if len(layer_dims) < 2:
        raise ValueError("The length of layers must be at least 2 (Input -> Output)") 
    
    if min(layer_dims) < 1:
          raise ValueError("The number of neurons in every layer must be at least 1")
    
    if len(layer_dims) != len(activations) + 1:
          raise ValueError(
              f"Structure Error: You provided {len(layer_dims)} layers but {len(activations)} activations. "
              f"Expected {len(layer_dims) - 1} activations."
          )

    valid_activations = {"relu", "sigmoid", "softmax"}
    for act in activations:
        if act not in valid_activations:
            raise ValueError(f"Invalid activation '{act}'. Supported: {valid_activations}")

    valid_losses = {"mse", "bce", "cce"}
    if loss_type not in valid_losses:
        raise ValueError(f"Invalid loss_type '{loss_type}'. Supported: {valid_losses}")

In [17]:
nn = NeuralNetwork(
  activations=["relu", "relu", "relu", "softmax"],
  layer_dims=[3, 4, 2, 3, 2],
  loss_type="mse",
  optimizer_type="adam",
)

In [18]:
nn.initialize_parameters()

In [21]:
nn.params

{'W1': array([[ 0.47891111,  0.26608096, -0.70765682],
        [-0.91507523, -0.04087161,  0.89699431],
        [ 1.63706976, -0.2084502 ,  0.01910965],
        [ 0.1930621 ,  0.20960233, -0.4003634 ]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.]]),
 'W2': array([[ 0.17098864,  1.13935507,  0.34501736, -0.19191044],
        [ 0.13064075,  1.66551233, -1.26792106, -0.63924418]]),
 'b2': array([[0.],
        [0.]]),
 'W3': array([[ 0.2872596 , -0.75923395],
        [ 1.80214212, -0.83847533],
        [ 0.18337879, -1.06952569]]),
 'b3': array([[0.],
        [0.],
        [0.]]),
 'W4': array([[ 0.47207236, -0.35167989,  0.24454039],
        [-0.24634172, -0.20869113,  0.17367285]]),
 'b4': array([[0.],
        [0.]])}